# TFIDF


<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

## Task-1

<font face='georgia'>
    <h4><strong>1. Build a TFIDF Vectorizer & compare its results with Sklearn:</strong></h4>

<ul>
    <li> As a part of this task you will be implementing TFIDF vectorizer on a collection of text documents.</li>
    <br>
    <li> You should compare the results of your own implementation of TFIDF vectorizer with that of sklearns implemenation TFIDF vectorizer.</li>
    <br>
    <li> Sklearn does few more tweaks in the implementation of its version of TFIDF vectorizer, so to replicate the exact results you would need to add following things to your custom implementation of tfidf vectorizer:
       <ol>
        <li> Sklearn has its vocabulary generated from idf sroted in alphabetical order</li>
        <li> Sklearn formula of idf is different from the standard textbook formula. Here the constant <strong>"1"</strong> is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions.
            
 $IDF(t) = 1+\log_{e}\frac{1\text{ }+\text{ Total  number of documents in collection}} {1+\text{Number of documents with term t in it}}.$
        </li>
        <li> Sklearn applies L2-normalization on its output matrix.</li>
        <li> The final output of sklearn tfidf vectorizer is a sparse matrix.</li>
    </ol>
    <br>
    <li>Steps to approach this task:
    <ol>
        <li> You would have to write both fit and transform methods for your custom implementation of tfidf vectorizer.</li>
        <li> Print out the alphabetically sorted voacb after you fit your data and check if its the same as that of the feature names from sklearn tfidf vectorizer. </li>
        <li> Print out the idf values from your implementation and check if its the same as that of sklearns tfidf vectorizer idf values. </li>
        <li> Once you get your voacb and idf values to be same as that of sklearns implementation of tfidf vectorizer, proceed to the below steps. </li>
        <li> Make sure the output of your implementation is a sparse matrix. Before generating the final output, you need to normalize your sparse matrix using L2 normalization. You can refer to this link https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html </li>
        <li> After completing the above steps, print the output of your custom implementation and compare it with sklearns implementation of tfidf vectorizer.</li>
        <li> To check the output of a single document in your collection of documents,  you can convert the sparse matrix related only to that document into dense matrix and print it.</li>
        </ol>
    </li>
    <br>
   </ul>

 

### Corpus

In [25]:
import tqdm
import math
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize

In [15]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

In [11]:
def tfidf_fit(dataset):
    unique_words = set() # at first we will initialize an empty set
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        
        return vocab
    else:
        print("you need to pass list of sentance")
        
vocab = tfidf_fit(corpus)
print("the vocab from TFIDF fit is ")
print(list(vocab.keys()))

the vocab from TFIDF fit is 
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [34]:
        
def idf_computation(dataset):
    rows = []
    columns = []
    values = []
    word_doc_dict = dict()
    word_idf_dict = dict()
    number_documents = len(dataset)
    for word in vocab:
        wc = 0
        for idx, row in enumerate(dataset): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_list = row.split(' ')
            if word in word_list:
                wc += 1
        
        word_idf_dict[word] = round(1+(math.log((1+number_documents)/(1+wc))),8)
    return word_idf_dict

def tfidf_transform(dataset):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(dataset): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            number_words = len(row.split())
            # for every unique word in the document
            for word, freq in word_freq.items():  # for each unique word in the review.                
                if len(word) < 2 or word not in vocab.keys():
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                tf_value = word_freq[word]/number_words
                tfidf_value = tf_value * idf_dict[word]
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the frequency of the word
                    values.append(tfidf_value)
        return csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
    else:
        print("you need to pass list of strings")
    
      
    
        
# tf_dict = tf_computation(dataset,vocab)
idf_dict = idf_computation(corpus)
print("the IDF value  are ")
print(idf_dict)
sparse_matrix = tfidf_transform(corpus)

the IDF value  are 
{'and': 1.91629073, 'document': 1.22314355, 'first': 1.51082562, 'is': 1.0, 'one': 1.91629073, 'second': 1.91629073, 'the': 1.0, 'third': 1.91629073, 'this': 1.0}


In [40]:
print(sparse_matrix)

  (0, 1)	0.24462871000000003
  (0, 2)	0.30216512400000006
  (0, 3)	0.2
  (0, 6)	0.2
  (0, 8)	0.2
  (1, 1)	0.4077145166666667
  (1, 3)	0.16666666666666666
  (1, 5)	0.3193817883333333
  (1, 6)	0.16666666666666666
  (1, 8)	0.16666666666666666
  (2, 0)	0.3193817883333333
  (2, 3)	0.16666666666666666
  (2, 4)	0.3193817883333333
  (2, 6)	0.16666666666666666
  (2, 7)	0.3193817883333333
  (2, 8)	0.16666666666666666
  (3, 1)	0.24462871000000003
  (3, 2)	0.30216512400000006
  (3, 3)	0.2
  (3, 6)	0.2
  (3, 8)	0.2


In [41]:
sparse_matrix_l2_normalised = csr_matrix(normalize(spare_matrix.toarray(),norm = 'l2'))

In [46]:
print(sparse_matrix_l2_normalised)

  (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149
  (1, 1)	0.6876235979836937
  (1, 3)	0.2810886740337529
  (1, 5)	0.5386476208856762
  (1, 6)	0.2810886740337529
  (1, 8)	0.2810886740337529
  (2, 0)	0.511848512707169
  (2, 3)	0.267103787642168
  (2, 4)	0.511848512707169
  (2, 6)	0.267103787642168
  (2, 7)	0.511848512707169
  (2, 8)	0.267103787642168
  (3, 1)	0.4697913855799205
  (3, 2)	0.580285823684436
  (3, 3)	0.3840852409148149
  (3, 6)	0.3840852409148149
  (3, 8)	0.3840852409148149


In [43]:
print(sparse_matrix_l2_normalised[0])

  (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149


### SkLearn Implementation

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [10]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [ ]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [ ]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [ ]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [ ]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


## Task-2

<font face='georgia'>
    <h4><strong>2. Implement max features functionality:</strong></h4>

<ul>
    <li> As a part of this task you have to modify your fit and transform functions so that your vocab will contain only 50 terms with top idf scores.</li>
    <br>
    <li>This task is similar to your previous task, just that here your vocabulary is limited to only top 50 features names based on their idf values. Basically your output will have exactly 50 columns and the number of rows will depend on the number of documents you have in your corpus.</li>
    <br>
    <li>Here you will be give a pickle file, with file name <strong>cleaned_strings</strong>. You would have to load the corpus from this file and use it as input to your tfidf vectorizer.</li>
    <br>
    <li>Steps to approach this task:
    <ol>
        <li> You would have to write both fit and transform methods for your custom implementation of tfidf vectorizer, just like in the previous task. Additionally, here you have to limit the number of features generated to 50 as described above.</li>
        <li> Now sort your vocab based in descending order of idf values and print out the words in the sorted voacb after you fit your data. Here you should be getting only 50 terms in your vocab. And make sure to print idf values for each term in your vocab. </li>
        <li> Make sure the output of your implementation is a sparse matrix. Before generating the final output, you need to normalize your sparse matrix using L2 normalization. You can refer to this link https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html </li>
        <li> Now check the output of a single document in your collection of documents,  you can convert the sparse matrix related only to that document into dense matrix and print it. And this dense matrix should contain 1 row and 50 columns. </li>
        </ol>
    </li>
    <br>
   </ul>

In [47]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


In [48]:
corpus

['slow moving aimless movie distressed drifting young man',
 'not sure lost flat characters audience nearly half walked',
 'attempting artiness black white clever camera angles movie disappointed became even ridiculous acting poor plot lines almost non existent',
 'little music anything speak',
 'best scene movie gerardo trying find song keeps running head',
 'rest movie lacks art charm meaning emptiness works guess empty',
 'wasted two hours',
 'saw movie today thought good effort good messages kids',
 'bit predictable',
 'loved casting jimmy buffet science teacher',
 'baby owls adorable',
 'movie showed lot florida best made look appealing',
 'songs best muppets hilarious',
 'cool',
 'right case movie delivers everything almost right face',
 'average acting main person low budget clearly see',
 'review long overdue since consider tale two sisters single greatest film ever made',
 'put gem movie terms screenplay cinematography acting post production editing directing aspect film makin

In [86]:
def tfidf_fit(dataset):
    unique_words = set() # at first we will initialize an empty set
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        
        return vocab
    else:
        print("you need to pass list of sentance")




def corpus_idf_computation(dataset):
    rows = []
    columns = []
    values = []
    word_doc_dict = dict()
    word_idf_dict = dict()
    number_documents = len(dataset)
    for word in vocab:
        wc = 0
        for idx, row in enumerate(dataset): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_list = row.split(' ')
            if word in word_list:
                wc += 1
        
        word_idf_dict[word] = round(1+(math.log((1+number_documents)/(1+wc))),8)
    return word_idf_dict



def get_top_idf(idf_dict):
    
    idf_dict_sorted_list = sorted(idf_dict.items(), key=lambda x: x[1],reverse = True)    
    idf_dict_sorted = dict(idf_dict_sorted_list[:50])
    print("the number of keys is ",len(idf_dict_sorted.keys()))
#     print(idf_dict_sorted)
    return idf_dict_sorted

def filter_vocab(vocab,top_50_idf):
    latest_vocab = []
    vocab_dict = dict()
    for word in vocab:
        if word in top_50_idf.keys():
            latest_vocab.append(word)
    
    latest_vocab.sort()        
    for i,j in enumerate(latest_vocab):
        vocab_dict[j] = i
    
    return vocab_dict

def tfidf_transform(dataset,vocab):
    rows = []
    columns = []
    values = []
    if isinstance(dataset, (list,)):
        for idx, row in enumerate(dataset): # for each document in the dataset
            # it will return a dict type object where key is the word and values is its frequency, {word:frequency}
            word_freq = dict(Counter(row.split()))
            # for every unique word in the document
            number_words = len(row.split())
            # for every unique word in the document
            for word, freq in word_freq.items():  # for each unique word in the review.                
                if len(word) < 2 or word not in vocab.keys():
                    continue
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
                tf_value = word_freq[word]/number_words
                tfidf_value = tf_value * idf_dict[word]
                col_index = vocab.get(word, -1) # retreving the dimension number of a word
                # if the word exists
                if col_index !=-1:
                    # we are storing the index of the document
                    rows.append(idx)
                    # we are storing the dimensions of the word
                    columns.append(col_index)
                    # we are storing the frequency of the word
                    values.append(tfidf_value)
        return csr_matrix((values, (rows,columns)), shape=(len(dataset),len(vocab)))
    else:
        print("you need to pass list of strings")
    
        

In [94]:
vocab = tfidf_fit(corpus)
print("number of unique words in the vocab is ",len(vocab.keys()))
idf_dict = corpus_idf_computation(corpus)
top_50_idf = get_top_idf(idf_dict)
print("TOP 50 vocab based on IDF scores")
print(top_50_idf)
latest_vocab = filter_vocab(vocab,top_50_idf)
sparse_matrix_rep = tfidf_transform(corpus,latest_vocab)
final_sparse_matrix_l2_normalised = csr_matrix(normalize(sparse_matrix_rep.toarray(),norm = 'l2'))

number of unique words in the vocab is  2886
the number of keys is  50
TOP 50 vocab based on IDF scores
{'aailiyah': 6.922918, 'abandoned': 6.922918, 'abroad': 6.922918, 'abstruse': 6.922918, 'academy': 6.922918, 'accents': 6.922918, 'accessible': 6.922918, 'acclaimed': 6.922918, 'accolades': 6.922918, 'accurate': 6.922918, 'accurately': 6.922918, 'achille': 6.922918, 'ackerman': 6.922918, 'actions': 6.922918, 'adams': 6.922918, 'add': 6.922918, 'added': 6.922918, 'admins': 6.922918, 'admiration': 6.922918, 'admitted': 6.922918, 'adrift': 6.922918, 'adventure': 6.922918, 'aesthetically': 6.922918, 'affected': 6.922918, 'affleck': 6.922918, 'afternoon': 6.922918, 'aged': 6.922918, 'ages': 6.922918, 'agree': 6.922918, 'agreed': 6.922918, 'aimless': 6.922918, 'aired': 6.922918, 'akasha': 6.922918, 'akin': 6.922918, 'alert': 6.922918, 'alike': 6.922918, 'allison': 6.922918, 'allow': 6.922918, 'allowing': 6.922918, 'alongside': 6.922918, 'amateurish': 6.922918, 'amaze': 6.922918, 'amazed': 

In [91]:
sparse_matrix_rep

<746x50 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Compressed Sparse Row format>

In [92]:
final_sparse_matrix_l2_normalised

<746x50 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Compressed Sparse Row format>

In [104]:
print(final_sparse_matrix_l2_normalised)

  (0, 30)	1.0
  (68, 24)	1.0
  (72, 29)	1.0
  (74, 31)	1.0
  (119, 33)	1.0
  (135, 3)	0.3779644730092272
  (135, 10)	0.3779644730092272
  (135, 18)	0.3779644730092272
  (135, 20)	0.3779644730092272
  (135, 36)	0.3779644730092272
  (135, 40)	0.3779644730092272
  (135, 41)	0.3779644730092272
  (176, 49)	1.0
  (181, 13)	1.0
  (192, 21)	1.0
  (193, 23)	1.0
  (216, 2)	1.0
  (222, 47)	1.0
  (225, 19)	1.0
  (227, 17)	1.0
  (241, 44)	1.0
  (270, 1)	1.0
  (290, 25)	1.0
  (333, 26)	1.0
  (334, 15)	1.0
  (341, 43)	1.0
  (344, 42)	1.0
  (348, 8)	1.0
  (377, 37)	1.0
  (409, 5)	1.0
  (430, 39)	1.0
  (457, 45)	1.0
  (461, 4)	1.0
  (465, 38)	1.0
  (475, 35)	1.0
  (493, 6)	1.0
  (500, 48)	1.0
  (548, 0)	0.7071067811865475
  (548, 32)	0.7071067811865475
  (608, 14)	1.0
  (612, 11)	1.0
  (620, 46)	1.0
  (632, 7)	1.0
  (644, 12)	0.7071067811865475
  (644, 27)	0.7071067811865475
  (664, 28)	1.0
  (667, 22)	1.0
  (691, 34)	1.0
  (697, 9)	1.0
  (722, 16)	1.0


In [107]:
print(final_sparse_matrix_l2_normalised.toarray()[135])

[0.         0.         0.         0.37796447 0.         0.
 0.         0.         0.         0.         0.37796447 0.
 0.         0.         0.         0.         0.         0.
 0.37796447 0.         0.37796447 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.37796447 0.         0.         0.         0.37796447 0.37796447
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
